In [2]:
import numpy as np
import six
import sys
import os
import traceback
import re
import pickle


from chainer import cuda
from context2vec.common.context_models import Toks
from context2vec.common.model_reader import ModelReader
import sklearn
import pandas as pd
import logging
from scipy.stats import spearmanr
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import gensim


[nltk_data] Downloading package stopwords to /home/ql261/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
# def space_punc(sent):
#     new_sent=sent
#     add=0
#     for m in re.finditer(r'[\d\-\/\"\';,.!?\)\(]+',sent):
#         if m.end(0)+add>=len(new_sent):
#             return new_sent[:m.start(0)+add]+' '+new_sent[m.start(0)+add:m.end(0)+add]
#         else:
#             new_sent=new_sent[:m.start(0)+add]+' '+new_sent[m.start(0)+add:m.end(0)+add]+' '+new_sent[m.end(0)+add:]
#             add+=2
#     return new_sent
        
# def mult_sim(w, target_v, context_v):
#     target_similarity = w.dot(target_v)
#     target_similarity[target_similarity<0] = 0.0
#     context_similarity = w.dot(context_v)
#     context_similarity[context_similarity<0] = 0.0
#     return (target_similarity * context_similarity)


# def find_unk_pos(words,test_w):
#     try:
#         pos=words.index(test_w.upper())
#     except ValueError:
#         try:
#             pos=words.index(test_w.upper()+'S')
#         except ValueError as e:
#             try:
#                 pos=words.index(test_w.upper()+'ES')
#             except ValueError as e:
#                 try:
#                     pos=words.index(test_w.upper()[:-1]+'IES')
#                 except ValueError as e:
#                     try:
#                         pos=words.index(test_w.upper()+test_w.upper()[-1]+'ES')
#                     except ValueError as e:
#                         print (e)
#     return pos

def produce_top_n_simwords(w_filter,context_embed,n_result,index2word):
        # compute top n_result similarity weights

        print('producing top {0} simwords'.format(n_result))
        similarity = (w_filter.dot(context_embed)+1.0)/2
        top_words_i=[]
        count = 0
        for i in (-similarity).argsort():
                    if xp.isnan(similarity[i]):
                        continue
                    print('{0}: {1}'.format(str(index2word[i]), str(similarity[i])))
                    count += 1
                    top_words_i.append(i)
                    if count == n_result:
                        break

        top_vec=w_filter[top_words_i,:]
        return top_vec
def skipgram_context(model,words,pos):
    context_wvs=[]
    for i,word in enumerate(words):
        if i != pos:
            try:
                context_wvs.append(model[word])
                
            except KeyError as e:
                print ('==warning==: key error in context {0}'.format(e))
    context_embed=sum(np.array(context_wvs))/len(context_wvs)
    return len(context_wvs),context_embed
    
def context_inform(test_s,test_w, model,model_type,n_result,w_filter,index2word,weight):
    test_s=test_s.replace(test_w, ' '+test_w+' ')

    #test_s=space_punc(test_s)
    print(test_s)
    words=test_s.split()
    pos=words.index(test_w)
    
    score=1.0
    #decide on model
    if model_type=='context2vec':
        context_embed= model.context2vec(words, pos)
        context_embed = context_embed / xp.sqrt((context_embed * context_embed).sum())
    
    elif model_type=='skipgram':
        score,context_embed=skipgram_context(model,words,pos)
        context_embed = context_embed / xp.sqrt((context_embed * context_embed).sum())
        
    else:
        print ('model type {0} not recognized'.format(model_type))
        sys.exit(1)
        
        
    #decide on weight 
    if weight=='top_mutual_sim':
        top_vec=produce_top_n_simwords(w_filter,context_embed,n_result,index2word)
        score=sum(sum(top_vec.dot(top_vec.T)))/(n_result**2)
    elif weight=='learned':
        print ('learned not implemented')
    elif weight=='gaussian':
        print ('gaussian not implemented')
    elif weight ==False:
        score=score
    else:
        print ('weight mode {0} not recognized'.format(weight))
    return score,context_embed

def additive_model(test_ss,test_w, model_type,model,n_result,w_filter,index2word,weight=False):
    context_out=[]
    context_weights=[]
    for test_s in test_ss.split('@@'):
        test_s=test_s.strip()
        score,context_embed=context_inform(test_s,test_w, model,model_type,n_result,w_filter,index2word,weight)
        print ('weight is {0}'.format(score))
        context_out.append(context_embed)
        context_weights.append(score)
    context_out=np.array(context_out)
    norm_weights=np.array(context_weights).reshape(len(context_weights),1)/float(sum(context_weights))
    print ('normalized weight: \n  {0}'.format(norm_weights))
    context_avg=sum(norm_weights*context_out)
    print('producing top {0} words for new embedding'.format(n_result))
    top_vec=produce_top_n_simwords(w_filter,context_avg,n_result,index2word)
     
    return context_avg

def filter_w(w,word2index,index2word,word_freq_f):
    #filter out words with freq less than 200, words with no letters in, and stopwords
    stopw=stopwords.words('english')
    stopw=[word.encode('utf-8') for word in stopw]
    index2word_filter={}
    word2index_filter={}
    index_filter2index=[]
    counter=0
    with open(word_freq_f) as f:
        for line in f:
            f_w=line.split()
            if len(f_w)>1:
                if f_w[1] in word2index and re.search('[a-zA-Z]',f_w[1])!=None and f_w[1] not in stopw :
                    #word2freq_nostop[f_w[1]]=f_w[0]
                    index_filter2index.append(word2index[f_w[1]])
                    word2index_filter[f_w[1]]=counter
                    index2word_filter[counter]=f_w[1]
                    counter+=1
    w_filter= w[index_filter2index,:]
    return w_filter,word2index_filter,index2word_filter

def rm_stopw_context(model):
    stopw=stopwords.words('english')
    stopw=[word.encode('utf-8') for word in stopw]
    
    model={word:model.wv.__getitem__(word) for word in model.wv.vocab if word not in stopw}
    return model


def eval_chimera(chimeras_data_dir,num_sent,context_model,model_type,n_result,w,index2word,weight=False):
    golds=[]
    model_predict=[]
    num_indic='l'+str(num_sent)
    for root, subdir, fname in os.walk(chimeras_data_dir):
        for fn in fname:
            if fn.endswith('fixed.test.txt')and 'l'+str(num_sent)==fn.split('.')[1]: #read in the test file
                print (fn)
                data=pd.read_csv(os.path.join(chimeras_data_dir,fn),delimiter='\t')
                for index, row in data.iterrows():
                    
                    #compute context representation
                    if weight!='learned':
                        context_avg=additive_model(row[1],'___', model_type,context_model,n_result,w,index2word,weight)
                   
                    
                    #cosine similarity with probe embedding
                    for gold,probe in zip(row[3].split(','),row[2].split(',')):
                        try:
                            cos=w[word2index[probe]].dot(context_avg)
                            if xp.isnan(cos):
                                continue
                            else:
                                model_predict.append(cos)
                                golds.append(gold)
                        except KeyError as e:
                            print ("====warning key error for probe=====: {0}".format(e))
    if len(golds)==len(model_predict):
        print ('spearman correlation is {0}'.format(spearmanr(golds,model_predict)))
    else:
        print ('unequal length: gold {0}, model {1}'.format( len(golds),len(model_predict)))


    

In [25]:
import gensim



In [14]:
{key: model.wv.vocab[key].index for key in model.wv.vocab}

{'tripolitan': 151704,
 'biennials': 121973,
 'tsukino': 186810,
 'biysk': 256933,
 'woode': 193350,
 'verplank': 206266,
 'nunnery': 31611,
 'woodi': 253296,
 'gavar': 248756,
 'sowell': 65731,
 'utnapishtim': 169059,
 'fleeces': 139027,
 'woods': 3881,
 'clotted': 119004,
 'spiders': 11318,
 'gavan': 76144,
 'woody': 8886,
 'trawling': 52416,
 'comically': 43315,
 'gae': 117687,
 'spidery': 243024,
 'regularize': 138926,
 'caner': 188261,
 'yanagisawa': 133074,
 'canet': 99640,
 'fibrillose': 183122,
 'brodmann-1909': 227496,
 'sprague': 23072,
 'caney': 58478,
 'jairam': 132359,
 'kurigalzu': 241424,
 'pager': 65539,
 'yusaf': 235790,
 'chanthaburi': 143868,
 'igual': 177753,
 'fsos': 135740,
 'caned': 108096,
 'mirisch': 115073,
 'clottey': 170473,
 'rickman': 41966,
 'canek': 146088,
 'canem': 258149,
 'afikoman': 240201,
 'heliothis': 144725,
 'slothful': 218480,
 'naturopathic': 81903,
 'sidestrand': 232897,
 'grenadiers': 33291,
 'trojan': 15353,
 'pigment': 16806,
 'caryopsis'

In [29]:
model = gensim.models.Word2Vec.load('../models/wiki_all.model/wiki_all.sent.split.model')
stopw=stopwords.words('english')
stopw=[word.encode('utf-8') for word in stopw]
model={word:model.wv.__getitem__(word) for word in model.wv.vocab if word not in stopw}
model['it']

KeyError: 'it'

In [34]:

if __name__=="__main__":
    
    #params read in
    if sys.argv[0]=='/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py':
#         model_param_file='../models/context2vec/model_dir/context2vec.ukwac.model.params'
#         model_type='context2vec'
        
        model_param_file='../models/wiki_all.model/wiki_all.sent.split.model'
        model_type='skipgram'
        weight='top_mutual_sim'
        context_rm_stopw=1
        data='../data/data-chimeras'
    else:
        if len(sys.argv) < 3:
            print >> sys.stderr, "Usage: %s <model_param_file> <model_type> <weight> <context_rm_stop>"  % (sys.argv[0])
            sys.exit(1)

        model_param_file = sys.argv[1]
        model_type=sys.argv[2]
        weight=int(sys.argv[3])
        context_rm_stop=int.argv[4]
        data =argv[5]
    
    
    #gpu setup 
    gpu = -1 # todo: make this work with gpu

    if gpu >= 0:
        cuda.check_cuda_available()
        cuda.get_device(gpu).use()    
    xp = cuda.cupy if gpu >= 0 else np
    
    # logging
    logger = logging.getLogger()
    logger.setLevel(logging.DEBUG)

    
    #choose model type
    print ('read model....')
    if model_type=='context2vec':
        #read in model
        
        model_reader = ModelReader(model_param_file)
        w = model_reader.w
        index2word = model_reader.index2word
        word2index=model_reader.word2index
        model = model_reader.model
    elif model_type=='skipgram':
        model = gensim.models.Word2Vec.load(model_param_file)
        w=model.wv.vectors
        #vector normalize for probe w embedding
        s = np.sqrt((w * w).sum(1))
        s[s==0.] = 1.
        w /= s.reshape((s.shape[0], 1))
        
        index2word=model.wv.index2word
        word2index={key: model.wv.vocab[key].index for key in model.wv.vocab}
        
        
    #weight
    if weight=='top_mutual_sim':
        #filter w : remove stop words and low freq words 
        print ('filter words for target....')
        w,word2index,index2word=filter_w(w,word2index,index2word,'word_freq')
        n_result = 20
        
    # remove context stop words
    if int(context_rm_stopw)==1:
        print ('filter words for context....')

        model=rm_stopw_context(model)
        
    

read model....
filter words for target....
filter words for context....


In [33]:
model['it']

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


array([-0.07414069,  0.00237762, -0.01688991,  0.0833763 ,  0.01006094,
        0.00955694, -0.05165017,  0.08938711, -0.0160817 ,  0.03677505,
        0.07913393,  0.00544822,  0.00407244, -0.03488292,  0.09322681,
        0.05796747, -0.01086759, -0.0030441 , -0.03655259,  0.0709125 ,
        0.00300092,  0.04223266,  0.08857679, -0.08581661, -0.06407928,
       -0.08015744,  0.07483944,  0.03769429, -0.0359708 ,  0.00530221,
       -0.011898  ,  0.03454955, -0.04965986, -0.13030227, -0.12247992,
       -0.02093698,  0.06013397,  0.03502217,  0.02312816,  0.03136777,
        0.10330871, -0.09440369, -0.07304221,  0.04065694, -0.02130606,
       -0.00402556, -0.03298032, -0.12454022, -0.05688494,  0.02294583,
       -0.04987674, -0.04972998, -0.01221978, -0.03869737, -0.0815587 ,
        0.01100218, -0.05448706,  0.03912188, -0.02788375, -0.0656127 ,
       -0.00192421,  0.03528941,  0.00329684, -0.02908289, -0.01353666,
       -0.01419983, -0.07041187, -0.04742733,  0.04589818,  0.01

In [38]:
#read in data
if data == '../data/data-chimeras':
        eval_chimera(data,6,model,model_type,n_result,w,index2word)

dataset.l6.fixed.test.txt
try snacking on slices of  ___  with yoghurt dip try low fat yoghurt mixed with chives and garlic
==warning==: key error in context 'on'
==warning==: key error in context 'of'
==warning==: key error in context 'with'
==warning==: key error in context 'with'
==warning==: key error in context 'and'
weight is 12
 ___  and parsnips are likely to be significant dietary sources of furocoumarins in the uk
==warning==: key error in context 'and'
==warning==: key error in context 'are'
==warning==: key error in context 'to'
==warning==: key error in context 'be'
==warning==: key error in context 'of'
==warning==: key error in context 'furocoumarins'
==warning==: key error in context 'in'
==warning==: key error in context 'the'
weight is 6
this is why i buy plenty at present a weeks shopping consists of at least 4  ___ 
==warning==: key error in context 'this'
==warning==: key error in context 'is'
==warning==: key error in context 'why'
==warning==: key error in contex

washbasin: 0.7406010073480171
kitchenette: 0.7212858634635917
kitchen: 0.7201230989548508
dinette: 0.7159585921746937
cloakrooms: 0.7128848345359409
utensils: 0.711110452187754
bedclothes: 0.709892580366653
cupboards: 0.708901733075461
rockeries: 0.7078107898416692
vacuumed: 0.7074745388291879
fridges: 0.7059742772831368
casseroles: 0.7050021705243321
croutons: 0.7045407013552234
bedsteads: 0.7041177055808345
tablecloths: 0.7032370149547821
laundry: 0.7031488086205805
utensil: 0.7027069045082779
soundproofed: 0.7023125809768553
cooking: 0.7018697071371951
drizzled: 0.7014043201572899
====warning key error for probe=====: 'wastebin'
doug whitney recorded in his logbooktakali blitzed 44  ___  on drome
==warning==: key error in context 'in'
==warning==: key error in context 'his'
==warning==: key error in context 'logbooktakali'
==warning==: key error in context 'on'
weight is 6
there are also important questions about the handling safeguarding and transport of  ___  systems
==warning==: 

myrtillus: 0.6765872159070455
kittiwakes: 0.6727672174561853
ruficollis: 0.668600106357061
cryptosporidiosis: 0.6672054068882527
moorhens: 0.6663402812742019
lapwings: 0.6660474318449271
approximately: 0.6657858252675137
freshwaters: 0.6649008438141567
hoofed: 0.6638276275545871
even: 0.6630780597365309
possibly: 0.662890183891469
trypanosome: 0.6628443664469734
30cm: 0.6621262199554037
180m: 0.6619207403808609
nectarines: 0.6615410906117412
animals: 0.6611663071062008
drylands: 0.6611248195885591
auks: 0.6608776032590659
apparently: 0.6607228420442528
1200m: 0.6599722984608326
====warning key error for probe=====: 'groundhog'
but his pleasure soon turns to distress when he sees that a baby  ___  is stuck in the mud and drowning
==warning==: key error in context 'but'
==warning==: key error in context 'his'
==warning==: key error in context 'to'
==warning==: key error in context 'when'
==warning==: key error in context 'he'
==warning==: key error in context 'that'
==warning==: key erro

saltmarshes: 0.7008867932736049
near: 0.7005945881211878
crannog: 0.6988349760940832
site: 0.6949602609820356
causewayed: 0.6930174669504584
myrtillus: 0.6924801208836528
reedbed: 0.692063490210047
grasslands: 0.6906469651093279
prehistoric: 0.690002938827292
scrubland: 0.6888160082536398
moorhens: 0.6874030937450791
heathlands: 0.6872162991000117
cropmarks: 0.6870754403598356
still: 0.6865849472009655
although: 0.6865627674889487
pollarded: 0.6862611336805347
probably: 0.6858412129168535
brochs: 0.6853632781861587
ostriches: 0.6849003491734476
lapwings: 0.6847090399404248
====warning key error for probe=====: 'groundhog'
this is a database of located and unlocated shipwrecks submarines aircraft single finds including a number of  ___  tusks
==warning==: key error in context 'this'
==warning==: key error in context 'is'
==warning==: key error in context 'a'
==warning==: key error in context 'of'
==warning==: key error in context 'and'
==warning==: key error in context 'unlocated'
==war

weight is 11
after his attack the beast confirmed itselfi live alone like a  ___ 
==warning==: key error in context 'after'
==warning==: key error in context 'his'
==warning==: key error in context 'the'
==warning==: key error in context 'itselfi'
==warning==: key error in context 'a'
weight is 6
and irresponsible comments of his foreign office led to the death of hapless tourists ___  watching
==warning==: key error in context 'and'
==warning==: key error in context 'of'
==warning==: key error in context 'his'
==warning==: key error in context 'to'
==warning==: key error in context 'the'
==warning==: key error in context 'of'
weight is 9
mathew was kind of like his name heavy set as a  ___  from the mountains of wyoming
==warning==: key error in context 'was'
==warning==: key error in context 'of'
==warning==: key error in context 'his'
==warning==: key error in context 'as'
==warning==: key error in context 'a'
==warning==: key error in context 'from'
==warning==: key error in contex

...

often: 0.6965192844956033
endearingly: 0.6952405485130266
make: 0.6950268553984866
mellifluous: 0.6944912330102146
ballsy: 0.6944000662553144
touchingly: 0.6919160468452201
unfussy: 0.6906323548529881
something: 0.6897073818576389
even: 0.6895752255908059
comping: 0.6895035602241789
entrancing: 0.689071525406723
captivating: 0.6880483858820607
soulful: 0.6880331346207389
psyched: 0.6878240081513928
floaty: 0.6877348370127332
soppy: 0.6875830641935962
engagingly: 0.687520955438105
glamourous: 0.6870688326510582
a widow whose arm was bitten off by a  ___  said yesterday she was sorry the creature was later killed
==warning==: key error in context 'a'
==warning==: key error in context 'was'
==warning==: key error in context 'off'
==warning==: key error in context 'by'
==warning==: key error in context 'a'
==warning==: key error in context 'she'
==warning==: key error in context 'was'
==warning==: key error in context 'the'
==warning==: key error in context 'was'
weight is 10
it has been e

psyched: 0.7025793709650702
strummed: 0.7003327096893526
saxes: 0.6995018743902695
strumming: 0.6993483141054566
saxophone: 0.6976936452902922
guitar: 0.6975957964164999
harmonica: 0.6967513238224344
bodhran: 0.6958650485581325
jangling: 0.695108973555153
glockenspiel: 0.6951025269666357
ohh: 0.6950155000540523
crooning: 0.6940627228214497
floaty: 0.692599136720055
melody: 0.692343569556766
synth: 0.6923131870080166
loping: 0.6916488403250515
bluesy: 0.6915925978884154
sounds: 0.6912130134946801
comping: 0.6901672897196205
vibraphone: 0.6900975512190776
art hallman played  ___  in the band and also doubled on vocals but the leading singer was the lovely georgia dey
==warning==: key error in context 'in'
==warning==: key error in context 'the'
==warning==: key error in context 'and'
==warning==: key error in context 'on'
==warning==: key error in context 'but'
==warning==: key error in context 'the'
==warning==: key error in context 'was'
==warning==: key error in context 'the'
weight i

drizzled: 0.7356613105847475
croutons: 0.7303842094688402
pureed: 0.7288649150079675
sugared: 0.7205908821311243
saucepan: 0.7199089816717114
vinaigrette: 0.7180909373575932
cutlets: 0.7155851205467978
washbasin: 0.7116588560111836
creamed: 0.7111962281682151
sipped: 0.7098251788260741
matchsticks: 0.7092741201450408
lidded: 0.7086653548918482
nightdress: 0.7085154767582549
shovelled: 0.7078192251169926
puree: 0.7075307570646315
chucked: 0.7073465202929573
wholemeal: 0.7063899481293374
fritters: 0.7063047660085524
browned: 0.7058072094991199
compote: 0.7055811605394643
be pleasant do nt carp but act like a  ___  and draw out all the information you can
==warning==: key error in context 'be'
==warning==: key error in context 'do'
==warning==: key error in context 'but'
==warning==: key error in context 'a'
==warning==: key error in context 'and'
==warning==: key error in context 'out'
==warning==: key error in context 'all'
==warning==: key error in context 'the'
==warning==: key error 

sweetcorn: 0.7767097162732441
vegetables: 0.7720168003538448
potatoes: 0.7688881168308725
cashews: 0.767105000949548
pureed: 0.76543665255172
beans: 0.7648113324796308
shallots: 0.7640613316892149
croutons: 0.7609374010395362
onions: 0.7593449155836289
gherkins: 0.7580458924300711
fritters: 0.7576531415247942
chickpeas: 0.7542052977041069
cutlets: 0.7530339284269848
puree: 0.7523721580284348
creamed: 0.7519842193038095
vinaigrette: 0.7513849710056706
lettuces: 0.7507200685464023
cassava: 0.7504049683870234
wholemeal: 0.7489503205657138
carrots: 0.7487715318844926
there was a foundry a tanner brick kiln two  ___  mills and two smithies a wheelwright and a saddler
==warning==: key error in context 'there'
==warning==: key error in context 'was'
==warning==: key error in context 'a'
==warning==: key error in context 'a'
==warning==: key error in context 'and'
==warning==: key error in context 'a'
==warning==: key error in context 'and'
==warning==: key error in context 'a'
weight is 10
ca

nightdress: 0.7070926551266243
moping: 0.7016333980430053
longingly: 0.7015604922875595
hypnotise: 0.6995979640620495
whimpering: 0.6988539142504492
peeing: 0.6966686955430648
moaned: 0.6960735727000547
trudging: 0.6959055359444631
forlornly: 0.6957379150212174
squints: 0.6946133472488742
sunburnt: 0.6944307070230242
mournfully: 0.6942310125737164
terrifies: 0.6942239672917643
cooped: 0.693740468778951
aunties: 0.6933124179347716
clambering: 0.6931226067780336
backsides: 0.692483341010802
sheepishly: 0.6924411113989635
dozed: 0.6923793777059991
sheepish: 0.6923642428401056
spearman correlation is SpearmanrResult(correlation=0.40842801898990466, pvalue=3.638999636240372e-26)
